# Bio-Inspired AI - Prey vs Predator

In [1]:
#from utils.utils_01.ga import generate_offspring
from random import choices, randrange, random, randint
from typing import List, Callable, Tuple
from collections import namedtuple
from functools import partial
import time

In [2]:
genome = List[int]
population = List[genome]
fitness_function = Callable[[genome], int]
populate_function = Callable[[], population]
selection_function = Callable[[population, fitness_function], Tuple[genome, genome]]
crossover_function = Callable[[genome, genome], Tuple[genome, genome]]
mutation_function = Callable[[genome, genome], genome]
Thing = namedtuple('thing', ['name', 'value', 'weight'])

In [3]:
things = [
    Thing('laptop', 500, 2000),
    Thing('headphones', 150, 160)
]

more_things = [
    Thing('mints', 5, 20),
    Thing('socks', 10, 30)
]

In [4]:
def generate_genome(length: int) -> genome:
    return choices([0, 1], k = length)

def generate_population(size: int, genome_length: int) -> population:
    return [generate_genome(genome_length) for _ in range(size)]

In [5]:
def fitness(genome: genome, things: List[Thing], weight_limit: int) -> int:
    if len(genome) != len(things):
        raise ValueError("Error")
    
    weight = 0
    value = 0

    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value

            if weight > weight_limit:
                return 0
    
    return value

In [6]:
def selection_pair(population: population, fitness_function: fitness_function) -> population:
    return choices(
        population = population,
        weights = [fitness_function(genome) for genome in population],
        k = 2
    )

In [7]:
def single_point_crossover(a: genome, b: genome) -> Tuple[genome, genome]:
    if len(a) != len(b):
        raise ValueError("Error")
    
    if len(a) < 2:
        return a, b
    
    p = randint(1, len(a) - 1)
    
    return a[0: p] + b[p:], b[0: p] + a[p:] 

In [8]:
def mutation(genome: genome, num: int = 1, probability: float = 0.5) -> genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
        return genome

In [9]:
def run_evolution(
        populate_func: populate_function,
        fitness_func: fitness_function,
        fitness_limit: int,
        selection_func: selection_function = selection_pair,
        crossover_func: crossover_function = single_point_crossover,
        mutation_func: mutation_function = mutation,
        generation_limit: int = 100
) -> Tuple[population, int]:
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(population, key = lambda genome: fitness_func(genome), reverse = True)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation
    
    population = sorted(population, key = lambda genome: fitness_func(genome), reverse = True)

    return population, i


In [13]:
start = time.time()

population, generations = run_evolution(
    populate_func = partial(
        generate_population, size = 10, genome_length = len(things)
    ),
    fitness_func = partial(
        fitness, things = things, weight_limit = 3000
    ), 
    fitness_limit = 740, 
    generation_limit = 100
)

end = time.time()

print("number of generations:", generations)
print("time passed:", end-start)

number of generations: 99
time passed: 0.01647782325744629
